In [ ]:
!pip install goatools

     |████████████████████████████████| 15.1 MB 154 kB/s 
     |████████████████████████████████| 149 kB 48.6 MB/s 
     |████████████████████████████████| 103 kB 58.4 MB/s 
  Created wheel for goatools: filename=goatools-1.1.6-py3-none-any.whl size=15762836 sha256=8adda7482e7efc0daed4859faea9656d9dec53f9abeed109745060da66ee9b11
  Stored in directory: /root/.cache/pip/wheels/ed/a7/4b/2e9ce970761985d8c6c6caab01fd0fa79fbc576942e11ad5e4
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9673 sha256=105e188385657364f7237416a9abbf921b381ad8cf8ebf5895ad25b14e52fd03
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built goatools wget
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
import pandas as pd
import numpy as np
import goatools
from goatools import obo_parser
import wget
import os
import json

In [ ]:
#loading the GenAge ageing genes dataset
genage_df = pd.read_csv('genage_models_export.tsv', delimiter = '\t', dtype={'Entrez Gene ID': object})

In [ ]:
#only retaining columns of interest
genage_df = genage_df.filter(['Gene ID', 'Entrez Gene ID', 'Gene Symbol', 'Ensembl ID', 'Longevity Influence'])

In [ ]:
#filtering out entries with missing/corrupted longevity labels
genage_df = genage_df[genage_df['Longevity Influence'].isin(['pro', 'anti']) == True]

In [ ]:
#dropping duplicate entries by all columns, keeping first entry
genage_df.drop_duplicates(keep = 'first', inplace = True)

In [ ]:
#selecting duplicate entries by Gene Symbol
slice_dups = genage_df[genage_df.duplicated(['Gene Symbol'], keep=False)]

In [ ]:
slice_dups.head(30)

,Gene ID,Entrez Gene ID,Gene Symbol,Ensembl ID,Longevity Influence
61,893,179246,atg-18,F41E6.13,pro
62,893,179246,atg-18,F41E6.13,anti
67,23,178005,atg-7,M7.5,pro
68,23,178005,atg-7,M7.5,anti
330,917,3564920,eak-7,K08E7.1,anti
331,917,3564920,eak-7,K08E7.1,pro
333,203,13183021,eat-18,NaN,anti
334,2066,24105308,eat-18,NaN,anti
339,210,172743,eef-2,F25H5.4,pro
340,210,172743,eef-2,F25H5.4,anti


In [ ]:
#excluding eat-18 as the Entrez IDs are different so manual review is needed to decide which one to keep
slice_dups = slice_dups[slice_dups['Gene Symbol'] != 'eat-18']

In [ ]:
#dropping duplicates by Gene Symbol, keeping first record in a duplicate pair
genage_df = genage_df[~genage_df['Gene Symbol'].isin(slice_dups['Gene Symbol'])]

In [ ]:
#removing one entry for eat-18 that links to a hypothetical protein
genage_df = genage_df[genage_df['Entrez Gene ID'] != '24105308']

In [ ]:
#dropping duplicates by Entrez Gene ID, also removing genes with no Entrez Gene ID
genage_df = genage_df.drop_duplicates(subset = ['Entrez Gene ID'], keep = 'first')
genage_df = genage_df.dropna(subset = ['Entrez Gene ID'])

In [ ]:
#record retained after initial clean-up
genage_df.shape

(855, 5)

In [ ]:
#obtaining an Entrez Gene IDs list to pass to BioMart
genage_df['Entrez Gene ID'].to_csv('genes.txt', sep = '\t', na_rep = 'N/A', index = False)

In [ ]:
#loading the QuickGO BP annotations dataset (biological process annotations with experimental and high-throughput evidence codes)
q = pd.read_csv('/content/QuickGO-annotations-EXPHT.tsv', delimiter = '\t', comment = '!')

In [ ]:
#loading the STRING protein aliases mapping file for C. elegans
string_map = pd.read_csv('/content/celegans.uniprot_2_string.2018.tsv', delimiter = '\t', header = None)

In [ ]:
#loading the Entrez to Uniprot mapping obtained via BioMart
j = pd.read_csv('/content/mart_export - UniProt.txt', delimiter = '\t', dtype={'NCBI gene (formerly Entrezgene) ID': object})

In [ ]:
#from the Entrez to Uniprot mapping, dropping duplicate entries (same Entrez Gene ID, same Swiss-Prot ID)
j = j.drop_duplicates(subset = ['NCBI gene (formerly Entrezgene) ID', 'UniProtKB/Swiss-Prot ID'], keep='first')

In [ ]:
#in the Entrez to Uniprot mapping, where an Entrez ID is not associated with a SwissProt code, use the TrEMBL code
j['UniProtKB/Swiss-Prot ID'] = j['UniProtKB/Swiss-Prot ID'].fillna(j['UniProtKB/TrEMBL ID'])

In [ ]:
#retaining the columns of interest in the STRING mapping file and renaming columns in preparation for merging
string_map.drop([0, 2, 4, 5], axis=1, inplace = True)
string_map.rename(columns = {1:'UniProtKB/Swiss-Prot ID', 3: 'STRING ID'}, inplace = True)

In [ ]:
#merging Entrez IDs/Uniprot IDs in the Biomart output with STRING IDs, to obtain STRING IDs of the ageing gene products
j_string = pd.merge(j, string_map, on = 'UniProtKB/Swiss-Prot ID')

In [ ]:
#dropping ageing genes that do not have an alias STRING ID, as they won't be present in the STRING PPI
j_string.dropna(subset = ['STRING ID'], inplace = True)

In [ ]:
#a new dataset, only retaining the ageing gene products with BP annotations
bp_j = j_string[j_string['UniProtKB/Swiss-Prot ID'].isin(q['GENE PRODUCT ID'])]

In [ ]:
#removing the TrEMBL ID column as it is redundant
bp_j.drop(['UniProtKB/TrEMBL ID'], axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#renaming the columns in the dataset of BP-annotated ageing gene codes in preparation for merging
bp_j.rename(columns = {'NCBI gene (formerly Entrezgene) ID':'Entrez Gene ID'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
genage_df_bp = genage_df[genage_df['Entrez Gene ID'].isin(bp_j['Entrez Gene ID'])]

In [ ]:
#merging the clean GenAge dataset with the dataset of BP-annotated ageing gene codes - the final dataset is the clean GenAge dataset, with associated UniProt and STRING ID codes for gene products
genage_uniprot = pd.merge(bp_j, genage_df_bp, on='Entrez Gene ID')

In [ ]:
#number of GenAge records retained (i.e. with gene products that are BP-annotated, identified by both UniProt IDs and STRING IDs)
genage_uniprot.shape

(388, 7)

In [ ]:
genage_uniprot['Longevity Influence'].value_counts()

anti    233
pro     155
Name: Longevity Influence, dtype: int64

In [ ]:
genage_uniprot.to_csv('genage_full.txt', index = False, sep = '\t')

In [ ]:
#for the QuickGO set of all C. elegans protein with BP, EXP and HTP-evidence annotations, map the STRING IDs using the STRING alias mapping file
string_map_2 = string_map.rename(columns = {'UniProtKB/Swiss-Prot ID': 'GENE PRODUCT ID'})
uniprot_to_string = pd.merge(q, string_map_2, on = 'GENE PRODUCT ID')

In [ ]:
#number of annotated proteins mapped to STRING IDs
uniprot_to_string['GENE PRODUCT ID'].nunique()

2980

In [ ]:
uniprot_to_string.head(20)

,GENE PRODUCT DB,GENE PRODUCT ID,SYMBOL,QUALIFIER,GO TERM,GO NAME,ECO ID,GO EVIDENCE CODE,REFERENCE,WITH/FROM,TAXON ID,ASSIGNED BY,ANNOTATION EXTENSION,GO ASPECT,STRING ID
0,UniProtKB,A0A0K3AR81,CELE_K06A5.1,involved_in,GO:0140543,positive regulation of piRNA transcription,ECO:0000315,IMP,PMID:34187893,NaN,6239,WB,NaN,P,6239.K06A5.1.2
1,UniProtKB,A0A0K3AR90,tom-1,involved_in,GO:0010807,regulation of synaptic vesicle priming,ECO:0001225,IMP,PMID:16895441,NaN,6239,SynGO,occurs_in(GO:0031594),P,6239.M01A10.2c
2,UniProtKB,A0A0K3AR90,tom-1,involved_in,GO:0010807,regulation of synaptic vesicle priming,ECO:0001225,IMP,PMID:16880125,NaN,6239,SynGO,NaN,P,6239.M01A10.2c
3,UniProtKB,A0A0K3AR90,tom-1,involved_in,GO:0010807,regulation of synaptic vesicle priming,ECO:0001225,IMP,PMID:16880125,NaN,6239,SynGO,occurs_in(GO:0031594),P,6239.M01A10.2c
4,UniProtKB,A0A0K3AR90,tom-1,involved_in,GO:0010807,regulation of synaptic vesicle priming,ECO:0006003,IDA,PMID:16880125,NaN,6239,SynGO,NaN,P,6239.M01A10.2c
5,UniProtKB,A0A0K3AR90,tom-1,involved_in,GO:0010807,regulation of synaptic vesicle priming,ECO:0006013,IDA,PMID:16895441,NaN,6239,SynGO,occurs_in(GO:0031594),P,6239.M01A10.2c
6,UniProtKB,A0A0K3AR90,tom-1,involved_in,GO:0010807,regulation of synaptic vesicle priming,ECO:0006013,IDA,PMID:16880125,NaN,6239,SynGO,NaN,P,6239.M01A10.2c
7,UniProtKB,A0A0K3AR90,tom-1,involved_in,GO:0010807,regulation of synaptic vesicle priming,ECO:0006013,IDA,PMID:16880125,NaN,6239,SynGO,occurs_in(GO:0031594),P,6239.M01A10.2c
8,UniProtKB,A0A0K3AR90,tom-1,involved_in,GO:0010807,regulation of synaptic vesicle priming,ECO:0006063,IMP,PMID:16880125,NaN,6239,SynGO,NaN,P,6239.M01A10.2c
9,UniProtKB,A0A0K3AR90,tom-1,involved_in,GO:0010807,regulation of synaptic vesicle priming,ECO:0006064,IDA,PMID:16880125,NaN,6239,SynGO,NaN,P,6239.M01A10.2c


In [ ]:
#dropping duplicate records by UniProt ID and GO term pairs, keeping first; dropping columns we are not interested in;
final_annos_BP = uniprot_to_string.drop_duplicates(subset = ['GENE PRODUCT ID', 'GO TERM'], keep = 'first')
final_annos_BP.drop(['GENE PRODUCT DB', 'QUALIFIER', 'GO NAME', 'ECO ID', 'GO EVIDENCE CODE', 'REFERENCE', 'WITH/FROM', 'TAXON ID', 'ASSIGNED BY', 'ANNOTATION EXTENSION', 'GO ASPECT'], axis=1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#creating a checkpoint annotations file
final_annos_BP.to_csv('quickgo_annos_BP.txt', sep = '\t', index = False)

In [ ]:
final_annos_BP.head()

,GENE PRODUCT ID,SYMBOL,GO TERM,STRING ID
0,A0A0K3AR81,CELE_K06A5.1,GO:0140543,6239.K06A5.1.2
1,A0A0K3AR90,tom-1,GO:0010807,6239.M01A10.2c
10,A0A0K3AR90,tom-1,GO:0099161,6239.M01A10.2c
13,A0A0K3AR90,tom-1,GO:0099148,6239.M01A10.2c
17,A0A0K3AUE4,sea-2,GO:0008340,6239.K10G6.3a


In [ ]:
go = obo_parser.GODag('go-basic.obo')

go-basic.obo: fmt(1.2) rel(2021-05-01) 47,284 GO Terms


In [ ]:
#propagating annotations, creating checkpoint file
propagation = {}
for term in final_annos_BP['GO TERM'].tolist():
  original = []
  original.append(term)
  rec = go[term]
  parents = list(rec.get_all_parents())
  propagation[term] = original+parents

with open('term_propagation.txt', 'w') as fp:
    fp.write(json.dumps(propagation))



In [ ]:
#mapping each original QuickGO annotation term to all ancestors obtained by propagation to root
final_annos_BP['Prop'] = final_annos_BP['GO TERM'].map(propagation)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
final_annos_BP.head()

,GENE PRODUCT ID,SYMBOL,GO TERM,STRING ID,Prop
0,A0A0K3AR81,CELE_K06A5.1,GO:0140543,6239.K06A5.1.2,"[GO:0140543, GO:0045893, GO:0019222, GO:003132..."
1,A0A0K3AR90,tom-1,GO:0010807,6239.M01A10.2c,"[GO:0010807, GO:0065007, GO:0050789, GO:004408..."
10,A0A0K3AR90,tom-1,GO:0099161,6239.M01A10.2c,"[GO:0099161, GO:0050794, GO:0060627, GO:005104..."
13,A0A0K3AR90,tom-1,GO:0099148,6239.M01A10.2c,"[GO:0099148, GO:0065007, GO:0060341, GO:005078..."
17,A0A0K3AUE4,sea-2,GO:0008340,6239.K10G6.3a,"[GO:0008340, GO:0032501, GO:0008150]"


In [ ]:
#creating a dictionary, with proteins identified by STRING IDs as keys and all annotations (original + propagations) as values
#saving to file for use in further experiments
group = (final_annos_BP.groupby('STRING ID')['Prop'].apply(list)).to_dict()
for key in group:
  l = group[key]
  flat_l = [item for sublist in l for item in sublist]
  group[key] = list(set(flat_l))

with open('prop_anno_STRING.txt', 'w') as fp:
    fp.write(json.dumps(group))